In [ ]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import BDay
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)


path = # FSI Indicators_Data

#############  # PLACEHOLDER FOR ALL DATES
dates_series = pd.date_range('2005-01-01', '2022-05-28', freq=BDay()).to_series()
dates_frame = dates_series.to_frame(name="Date")#.sort_values(by=['Date'], ascending=False)#.reset_index(drop=True)

########### LOAD DATA
    
USD_EUR = pd.read_csv(path + 'USD_EUR.csv')
USD_EUR = USD_EUR[['Date','Price']].rename(columns={"Price": "USD_EUR"})
USD_EUR['Date'] = pd.to_datetime(USD_EUR['Date'])
USD_EUR = USD_EUR.set_index('Date')

###########

USD_JPY = pd.read_csv(path + 'USD_JPY.csv')
USD_JPY = USD_JPY[['Date','Price']].rename(columns={"Price": "USD_JPY"})
USD_JPY['Date'] = pd.to_datetime(USD_JPY['Date'])
USD_JPY = USD_JPY.set_index('Date')

##########

CRED_INV= pd.read_csv(path + 'BAMLC0A0CM.csv')
CRED_INV = CRED_INV.rename(columns={"DATE": "Date" , "BAMLC0A0CM": "CRED_INV"})
CRED_INV['Date'] = pd.to_datetime(CRED_INV['Date'])
CRED_INV = CRED_INV.sort_values(by=['Date'], ascending=False)
CRED_INV = CRED_INV.set_index('Date')

##########

CRED_HY= pd.read_csv(path + 'BAMLHE00EHYIOAS.csv')
CRED_HY = CRED_HY.rename(columns={"DATE": "Date" , "BAMLHE00EHYIOAS": "CRED_HY"})
CRED_HY['Date'] = pd.to_datetime(CRED_HY['Date'])
CRED_HY = CRED_HY.sort_values(by=['Date'], ascending=False)
CRED_HY = CRED_HY.set_index('Date')

##########

TY_10 = pd.read_csv(path + 'TY_10.csv').dropna()
TY_10 = TY_10[['Date','Close']].rename(columns={"Close": "TY_10"})
TY_10['Date'] = pd.to_datetime(TY_10['Date'])
TY_10 = TY_10.sort_values(by=['Date'], ascending=False)
TY_10 = TY_10.set_index('Date')

##########

TY_30 = pd.read_csv(path + 'TY_30.csv').dropna()
TY_30 = TY_30[['Date','Close']].rename(columns={"Close": "TY_30"})
TY_30['Date'] = pd.to_datetime(TY_30['Date'])
TY_30 = TY_30.sort_values(by=['Date'], ascending=False)
TY_30 = TY_30.set_index('Date')

#########

SPYG = pd.read_csv(path + 'SPYG.csv').dropna()
SPYG = SPYG[['Date','Price']].rename(columns={"Price": "SPYG"})
SPYG['Date'] = pd.to_datetime(SPYG['Date'])
SPYG = SPYG.sort_values(by=['Date'], ascending=False)
SPYG = SPYG.set_index('Date')

#########

SPYV = pd.read_csv(path + 'SPYV.csv').dropna()
SPYV = SPYV[['Date','Price']].rename(columns={"Price": "SPYV"})
SPYV['Date'] = pd.to_datetime(SPYV['Date'])
SPYV = SPYV.sort_values(by=['Date'], ascending=False)
SPYV = SPYV.set_index('Date')

##########

WTI_USD = pd.read_csv(path + 'WTI_USD.csv').dropna()
WTI_USD = WTI_USD[['Date','Price']].rename(columns={"Price": "WTI_USD"})
WTI_USD['Date'] = pd.to_datetime(WTI_USD['Date'])
WTI_USD = WTI_USD.sort_values(by=['Date'], ascending=False)
WTI_USD = WTI_USD.set_index('Date')

##########

XAU_USD = pd.read_csv(path + 'XAU_USD.csv',  thousands=',').dropna()
XAU_USD = XAU_USD[['Date','Price']].rename(columns={"Price": "XAU_USD"})
XAU_USD['Date'] = pd.to_datetime(XAU_USD['Date'])
XAU_USD = XAU_USD.sort_values(by=['Date'], ascending=False)
XAU_USD = XAU_USD.set_index('Date')

#########  Interpolate empty data

df = pd.concat([dates_frame, USD_EUR,USD_JPY, CRED_INV,CRED_HY,TY_10,TY_30,SPYG, SPYV, WTI_USD, XAU_USD], axis=1, join="outer")
df = df.dropna(subset=['Date'])
df = df.interpolate(method='linear', limit_direction='both')
df = df.reset_index(drop=True)


##################################################################### STANDARDIZE DATA
df_stand = df.copy()

features= ['USD_EUR', 'USD_JPY', 'CRED_INV', 'CRED_HY', 'TY_10', 'TY_30', 'SPYG', 'SPYV', 'WTI_USD', 'XAU_USD']


for col in features:
    col_lst=[]
    print(col)
    for i in range(0,len(df)):
            st_val = (df[col][i] - df[col].mean())/df[col].std()
            col_lst.append(st_val)
        
    df_stand[col] = col_lst

df_stand = df_stand.reset_index(drop=True)

##################################################################### STANDARDIZE DATA IN A ROLLING WINDOW OF 250 DAYS


lag = 250
df_stand_roll = df[lag:].copy()


for col in features:
    col_lst=[]
    print(col)
    for i in range(0,len(df)):
        j=i+lag
        if j<len(df):
            #print(j)
            st_val = (df[col][j] - df[col][i:j].mean())/df[col][i:j].std()
            col_lst.append(st_val)
        
    df_stand_roll[col] = col_lst

df_stand_roll = df_stand_roll.reset_index(drop=True)


################################################################ CHANGE SIGN FOR SOME INDICATORS
df_stand_sign = df_stand_roll.copy()
   
df_stand_sign['USD_EUR'] = - df_stand_sign['USD_EUR']
df_stand_sign['USD_JPY'] = - df_stand_sign['USD_JPY']
df_stand_sign['TY_10'] = - df_stand_sign['TY_10']
df_stand_sign['TY_30'] = - df_stand_sign['TY_30']
df_stand_sign['SPYG'] = - df_stand_sign['SPYG']
df_stand_sign['SPYV'] = - df_stand_sign['SPYV']
df_stand_sign['WTI_USD'] =  - df_stand_sign['WTI_USD']

################################################################ RUN PCA

lag2=10
features = ['USD_EUR', 'USD_JPY', 'CRED_INV', 'CRED_HY', 'TY_10', 'TY_30', 'SPYG', 'SPYV', 'WTI_USD', 'XAU_USD']

dfa = df_stand_sign.copy()
weights =[]
pdf =[]
for i in range(0,len(dfa)):
    j=i+lag2
    if j<= len(dfa):
    
    
        x = dfa.loc[0:j, features].values
   
        pca = PCA(n_components=1)
        principalComponents = pca.fit_transform(x)

        pdf.append(principalComponents[0])
        weights.append(pca.components_[0].tolist())
    
PCA_DF = pd.DataFrame(weights, columns = features)

PCA_DF['total_square'] = PCA_DF.USD_EUR**2 + PCA_DF.USD_JPY**2 + PCA_DF.CRED_INV**2 + PCA_DF.CRED_HY**2 + PCA_DF.TY_10**2 + PCA_DF.TY_30**2 + PCA_DF.SPYG**2 + PCA_DF.SPYV**2 + PCA_DF.WTI_USD**2 + PCA_DF.XAU_USD**2

PCA_DF['total'] = PCA_DF.USD_EUR + PCA_DF.USD_JPY + PCA_DF.CRED_INV + PCA_DF.CRED_HY + PCA_DF.TY_10 + PCA_DF.TY_30 + PCA_DF.SPYG + PCA_DF.SPYV + PCA_DF.WTI_USD + PCA_DF.XAU_USD

PCA_DF['Date'] = dfa['Date'][(lag2-1):].reset_index(drop=True)

df_stand_short = df_stand_sign[(len(df_stand_sign) - len(PCA_DF)):].reset_index(drop=True)

#######################   Multiply Weights with Standardize data ( Get PCAs)

df_mult = df_stand_short[features] * PCA_DF[features]
df_mult['Date']=PCA_DF['Date']
df_mult['total'] = df_mult[df_mult.columns[:-1]].sum(axis=1)

######################  AREA CHART

%matplotlib qt
dfg = df_mult.copy()
plt.figure(figsize=(30,8))
plt.stackplot(dfg.index, 
              [dfg['USD_EUR'], dfg['USD_JPY'], \
               dfg['CRED_INV'], dfg['CRED_HY'], \
               dfg['TY_10'], dfg['TY_30'],\
               dfg['SPYG'], dfg['SPYV'], \
               dfg['WTI_USD'],dfg['XAU_USD']], 
              labels=features,
              alpha=0.8)
w = list(range(0, len(dfg), 90))
labels = list(dfg['Date'].dt.strftime('%Y-%m-%d').iloc[w])
plt.xticks(ticks=w, labels = labels, rotation=90,fontsize=12 )
plt.title ('FSI - Covid19 Pandemic', fontsize=14 )
plt.yticks(fontsize=12 )
plt.ylabel('FSI', fontsize=12)
plt.legend(fontsize=14)
plt.show()

######################   LINE CHART - STANDARDIZED

%matplotlib qt
dfg = df_mult.copy()

import matplotlib.pyplot as plt
plt.figure(figsize=(30,8))

x1 = list(range(0,len(dfg.Date)))

SPYV = dfg.SPYV.to_list()
SPYG = dfg.SPYG.to_list()

USD_EUR = dfg.USD_EUR.to_list()
USD_JPY = dfg.USD_JPY.to_list()

TY_10 = dfg.TY_10.to_list()
TY_30 = dfg.TY_30.to_list()

CRED_INV = dfg.CRED_INV.to_list()
CRED_HY = dfg.CRED_HY.to_list()

XAU_USD = dfg.XAU_USD.to_list()
WTI_USD = dfg.WTI_USD.to_list()


###################

plt.plot(x1, USD_EUR, label = "USD_EUR", linewidth=5)
plt.plot(x1, USD_JPY, label = "USD_JPY", linewidth=5)

#plt.plot(x1, CRED_INV, label = "CRED_INV")
#plt.plot(x1, CRED_HY, label = "CRED_HY")

#plt.plot(x1, TY_10, label = "TY_10")
#plt.plot(x1, TY_30, label = "TY_30")

#plt.plot(x1, SPYG, label = "S&P 500 Growth", linewidth=5)
#plt.plot(x1, SPYV, label = "S&P 500 Value", linewidth=5)

#plt.plot(x1, XAU_USD, label = "XAU_USD", linewidth=5)
#plt.plot(x1, WTI_USD, label = "WTI_USD", linewidth=5)



w = list(range(0, len(dfg), 100))
labels = list(dfg['Date'].dt.strftime('%Y-%m-%d').iloc[w])
plt.xticks(ticks=w, labels = labels, rotation=90 , fontsize=10 )
plt.title ('Indicators in standardized values', fontsize=20 )
plt.yticks(fontsize=10 )

plt.legend(fontsize=14, loc =2)
plt.show()

#########################

%matplotlib qt

dff = df.copy()

import matplotlib.pyplot as plt
plt.figure(figsize=(30,8))

x1 = list(range(0,len(dff.Date)))

SPYV = dff.SPYV.to_list()
SPYG = dff.SPYG.to_list()
USD_EUR = dff.USD_EUR.to_list()
USD_JPY = dff.USD_JPY.to_list()
TY_10 = dff.TY_10.to_list()
TY_30 = dff.TY_30.to_list()
CRED_INV = dff.CRED_INV.to_list()
CRED_HY = dff.CRED_HY.to_list()
XAU_USD = dff.XAU_USD.to_list()
WTI_USD = dff.WTI_USD.to_list()

#plt.plot(x1, USD_EUR, label = "USD_EUR")
#plt.plot(x1, USD_JPY, label = "USD_JPY")

#plt.plot(x1, SPYG, label = "S&P 500 Growth")
#plt.plot(x1, SPYV, label = "S&P 500 Value")

plt.plot(x1, TY_10, label = "TY_10")
plt.plot(x1, TY_30, label = "TY_30")

#plt.plot(x1, CRED_INV, label = "CRED_INV")
#plt.plot(x1, CRED_HY, label = "CRED_HY")

#plt.plot(x1, XAU_USD, label = "XAU_USD")
#plt.plot(x1, WTI_USD, label = "WTI_USD")

w = list(range(0, len(dff), 100))
labels = list(dff['Date'].dt.strftime('%Y-%m-%d').iloc[w])
plt.xticks(ticks=w, labels = labels, rotation=90 , fontsize=12 )
plt.title ('Indicators in original values', fontsize=20 )
plt.yticks(fontsize=12 )
plt.ylabel('%', fontsize=20)

plt.legend(fontsize=14)
#plt.show()